In [1]:
import os
import csv
# import umap
import json
import librosa
import numpy as np
import matplotlib.pyplot as plt
# from sklearn import preprocessing
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import MinMaxScaler
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [11]:
import file_load
video_path = "./dataset"
name = ".mp3"

file_list, file_name, folder_list, folder_name = file_load.file_path(video_path, name)

for mp3, name in zip(file_list, file_name):
    save_path = mp3.replace('.mp3','')
    print(save_path)
    wave, sr = librosa.load(mp3,sr = 16000)
    audio_size = np.size(wave)
    print(wave.shape)
    split_size = 480000
    count = 0
    for split_point in range(0, audio_size, split_size):
        if count<1:
            split_audio = wave[split_point : split_point + split_size]
            np.save(save_path+'_{}'.format(count), split_audio)
            print("wave split {} OK".format(split_audio.shape))
        count += 1

./dataset/no8/Old School Turntables
(480654,)
wave split (480000,) OK
./dataset/no8/Say what’s on your mind
(480654,)
wave split (480000,) OK
./dataset/no8/NO THANXXX
(480654,)
wave split (480000,) OK
./dataset/no3/gogobebe
(480654,)
wave split (480000,) OK
./dataset/no3/Egotistic
(480654,)
wave split (480000,) OK
./dataset/no3/Angel
(480654,)
wave split (480000,) OK
./dataset/no4/Hollywood
(480654,)
wave split (480000,) OK
./dataset/no4/SF
(480654,)
wave split (480000,) OK
./dataset/no4/Lie
(480654,)
wave split (480000,) OK
./dataset/no2/Decalcomanie
(480654,)
wave split (480000,) OK
./dataset/no2/ILoveToo
(480654,)
wave split (480000,) OK
./dataset/no2/RainySeason
(480654,)
wave split (480000,) OK
./dataset/no10/44f4a79e49f7ffcec038573e94cdab5d1b98cda6
(480654,)
wave split (480000,) OK
./dataset/no10/6831b5cca17f371165e455369c736d0166336f36
(480654,)
wave split (480000,) OK
./dataset/no10/4224ae32a34215ee7e6e5d144dbe7f418a47e7b0
(480654,)
wave split (480000,) OK
./dataset/no7/wf
(480

In [16]:
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
import file_load

model_path = './wavenet-ckpt/wavenet-ckpt/model.ckpt-200000' #模型位置
npy_path = './dataset'#dataset位置
name = '.npy' #欲讀取的檔名

def wavenet_encode(file_path):
    audio = np.load(file_path)
    encoding = fastgen.encode(audio, model_path, len(audio))
    print(encoding.reshape((-1, 16)).shape)
    return encoding.reshape((-1, 16))

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for npy, name in zip(file_list, file_name):
    print(npy)
    wavenet_data = wavenet_encode(npy)
    std_wavenet = np.std(wavenet_data, axis=0)
    mean_wavenet = np.mean(wavenet_data, axis=0)
    
    average_difference_channels = np.zeros((16,))
    
    for i in range(0, len(wavenet_data) - 2, 2):
        temp = wavenet_data[i] - wavenet_data[i+1]
        average_difference_channels += temp
    average_difference_channels /= (len(wavenet_data) // 2)   
    average_difference_channels = np.array(average_difference_channels)
    
    concat_features_wavenet = np.hstack((std_wavenet, mean_wavenet))
    concat_features_wavenet = np.hstack((concat_features_wavenet, average_difference_channels))
    
    save_path = npy.replace('.npy', "") 
    print('save',concat_features_wavenet.shape)
    np.save(save_path+'_OK',concat_features_wavenet) #存特徵向量

./dataset/no8/Old School Turntables_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no8/NO THANXXX_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no8/Say what’s on your mind_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/Angel_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/gogobebe_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/Egotistic_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no4/Lie_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/n

In [9]:
import file_load
import numpy as np
npy_path = './dataset/'#dataset位置
name = 'OK.npy' #欲讀取的檔名

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for c in range(len(folder_name)):
    fi_list, fi_name, fo_list, fo_name = file_load.file_path(npy_path+'no'+str(c+1), name)
    npy_list = []
    name_list = []
    print('no'+str(c+1))
    print('')
    
    for npy, n in zip(fi_list, fi_name):
        vec = np.load(npy)
        npy_list.append(vec)
        name_list.append(n.replace('_0_OK', ''))
        print(n.replace('_0_OK', ''))
        
    if len(npy_list)==3:
        print('')
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist,'    {} 到 {}'.format(name_list[0], name_list[1]))
        dist = np.linalg.norm(npy_list[0]-npy_list[2])
        print(dist,'    {} 到 {}'.format(name_list[0], name_list[2]))
        dist = np.linalg.norm(npy_list[1]-npy_list[2])
        print(dist,'    {} 到 {}'.format(name_list[1], name_list[2]))
    
    elif len(npy_list)==2:
        print('')
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist)
    print("================")

no1

YOUTH
Bloom
MYMYMY

2.830699914456322     YOUTH 到 Bloom
2.27186095339642     YOUTH 到 MYMYMY
0.8675373922773715     Bloom 到 MYMYMY
no2

RainySeason
ILoveToo
Decalcomanie

1.3239942531157127     RainySeason 到 ILoveToo
1.8122381616397454     RainySeason 到 Decalcomanie
2.65248001066036     ILoveToo 到 Decalcomanie
no3

gogobebe
Egotistic
Angel

0.48026388253221775     gogobebe 到 Egotistic
1.788494752006492     gogobebe 到 Angel
1.5435984406194327     Egotistic 到 Angel
no4

SF
Lie
Hollywood

1.0652123327911913     SF 到 Lie
1.041392130835179     SF 到 Hollywood
0.6575773697672002     Lie 到 Hollywood
no5

EpikHigh-Decalcomanie
Colde-Lovestruck
EpikHigh-NOTHANXXX

1.7957430801751673     EpikHigh-Decalcomanie 到 Colde-Lovestruck
0.7965891389392296     EpikHigh-Decalcomanie 到 EpikHigh-NOTHANXXX
2.0336525322391497     Colde-Lovestruck 到 EpikHigh-NOTHANXXX
no6

Wolves
SidetoSide
petedavidson

1.6356742053229116     Wolves 到 SidetoSide
2.803708321277067     Wolves 到 petedavidson
1.2793955245668986